In [2]:
#1. a

In [3]:
#2.d

In [4]:
#3. b

In [7]:
#4. a

In [8]:
#5. a

In [9]:
#6. a

In [10]:
#7. d

In [11]:
#8. a

In [12]:
#9. b

In [13]:
#10. b

In [14]:
#11. d

In [16]:
import pandas as pd
import numpy as np
import sklearn.model_selection as Ms
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
df = pd.read_csv("C:/Users/HP/Desktop/energydata_complete.csv")

In [18]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [19]:
df.shape

(19735, 29)

In [21]:
# dropping the lights column and date column respectively
df.drop(['lights'], axis=1,inplace=True)

df.drop(['date'], axis=1, inplace=True)


In [25]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

normalized_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
# we now split the data into independent(X variable) and dependent(Y variable) variables

features_df = normalized_df.drop(['Appliances'], axis=1)

target = normalized_df['Appliances']
# splitting data into training and testing set

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features_df, target, test_size = 0.3, random_state = 42)

In [29]:
# initializing the model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

# fitting the model to the training dataset
linear_model.fit(x_train, y_train)

# deriving the predictions
predicted_values = linear_model.predict(x_test)

In [34]:
# 12
x1 = np.array(normalized_df['T2'])
y1 = np.array(normalized_df['T6'])
# reshaping 
x1 = x1.reshape(-1, 1)
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size = 0.3, random_state = 42)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x1_train, y1_train)
y1_pred = model.predict(x1_test)

from sklearn.metrics import r2_score
r2 = r2_score(y1_test, y1_pred)
round(r2, 2)

0.64

In [31]:
# 13
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2) 


0.05

In [32]:
# 14
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

In [33]:
# 15
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)


0.088

In [35]:
# 16
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.15

In [44]:
#to check the weights of the features using linear regression

def get_weights_df(model, feat, col_name):
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df


In [45]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)


Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [46]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)


Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [47]:
linear_model_weights = get_weights_df(linear_model, x_train, 'linear_model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')


In [48]:
final_weights

,Features,linear_model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.411071,-0.000000
1,T_out,-0.321860,-0.262172,0.000000
2,T2,-0.236178,-0.201397,0.000000
3,T9,-0.189941,-0.188916,-0.000000
4,RH_8,-0.157595,-0.156830,-0.000110
5,RH_out,-0.077671,-0.054724,-0.049557
6,RH_7,-0.044614,-0.045977,-0.000000
7,RH_9,-0.039800,-0.041367,-0.000000
8,T5,-0.015657,-0.019853,-0.000000
9,T1,-0.003281,-0.018406,0.000000


In [50]:
#17
#T_out, T3

In [70]:
#18

ridge_pred = ridge_reg.predict(x_test)
train_score = ridge_reg.score(x_train, y_train)
test_score = ridge_reg.score(x_test, y_test)

# Root Mean Square Error to 3 decimal places
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
round(rmse, 3)



0.088

In [ ]:
#19
#The answer is 4 from the lasso regression model above


In [69]:
#20


lasso_pred = lasso_reg.predict(x_test)
train_score = lasso_reg.score(x_train, y_train)
test_score = lasso_reg.score(x_test, y_test)


from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, lasso_pred)
rmse = np.sqrt(mse)
print('Root Mean Squared Error for ridge model: {}'.format(round(rmse, 3)))


Root Mean Squared Error for ridge model: 0.094
